EUREKA

i can use data algo project code to make new templates off the signal code and just change signal inputs
also, generate an email each time
also, generate the code for stocks, etf, and forex and futures!

simply alter code for oanda and use this template, perfect it for a backtest and simply convert to forward test when ready to 
try it in a live market but fake account, when ready for real account, simply add oanda account info etc and key

continue to develop the code so it runs cleanly and generate solid seaborn graphics, add a scheduler for forward test and live deployment. 

oct 3, 2023 7 58 pm 
good work on a late start 

In [18]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import pandas as pd
from datetime import datetime, timedelta
import time
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import numpy as np
import matplotlib.pyplot as plt
import warnings 
# Ignore all warnings
warnings.filterwarnings('ignore')

In [19]:

# OANDA API credentials
api_token = '4774a622bc314440290f530efcc46453-eb96728988593ba944d5285fbcf8786a'
account_id = '101-001-8028197-001'

# OANDA API connection
api = oandapyV20.API(access_token=api_token)

In [20]:
# Download Data for Backtest 
client = oandapyV20.API(access_token=api_token)

# Get historical data for EURUSD
instrument = "EUR_USD"
start_date = "2023-06-01"
end_date = "2023-10-03"
params = {"from": start_date, "to": end_date, "granularity": "D"}
r = instruments.InstrumentsCandles(instrument=instrument, params=params)
candles = client.request(r)["candles"]

# Create a DataFrame from the historical data
df = pd.DataFrame([{"time": c["time"], "open": c["mid"]["o"], "high": c["mid"]["h"], "low": c["mid"]["l"], "close": c["mid"]["c"]} for c in candles])
df["time"] = pd.to_datetime(df["time"])

display(df)

,time,open,high,low,close
0,2023-05-31 21:00:00+00:00,1.06854,1.07685,1.06618,1.07622
1,2023-06-01 21:00:00+00:00,1.07643,1.07790,1.07052,1.07078
2,2023-06-04 21:00:00+00:00,1.07086,1.07224,1.06746,1.07119
3,2023-06-05 21:00:00+00:00,1.07161,1.07329,1.06670,1.06922
4,2023-06-06 21:00:00+00:00,1.06936,1.07398,1.06684,1.06985
...,...,...,...,...,...
84,2023-09-26 21:00:00+00:00,1.05698,1.05743,1.04880,1.05026
85,2023-09-27 21:00:00+00:00,1.05074,1.05790,1.04911,1.05664
86,2023-09-28 21:00:00+00:00,1.05616,1.06171,1.05581,1.05744
87,2023-10-01 21:00:00+00:00,1.05720,1.05919,1.04779,1.04780


In [21]:
# Calculate 10-day moving average of the open prices
df['MA10'] = df['open'].rolling(window=10).mean()

# Convert Data types for easy integration
df['MA10'] = df['MA10'].astype(float)
df['open'] = df['open'].astype(float)
df['high'] = df['high'].astype(float)
df['low'] = df['low'].astype(float)
df['close'] = df['close'].astype(float)

# Create a 'Signal' column indicating when the short moving average crosses the long.
df['Signal'] = np.where(df['open'] > df['MA10'], 1.0, 0.0)

# Create a new column named "Order", buy the day after (diff) when signal is generated, buy on the open, sell at the close! 
df["StratOrder"] = ""
StratOrder = df['StratOrder']
df.loc[df["Signal"] == 1.0, "StratOrder"] = "Buy"

df['Signal'] = 0  # 0 represents no signal
df.loc[df['open'] < df['MA10'], 'Signal'] = -1  # Sell signal
df.loc[df['open'] > df['MA10'], 'Signal'] = 1   # Buy signal

# Drop null values 
df = df.dropna()

In [22]:
display(df)

,time,open,high,low,close,MA10,Signal,StratOrder
9,2023-06-13 21:00:00+00:00,1.07954,1.08644,1.07740,1.08316,1.073528,1,Buy
10,2023-06-14 21:00:00+00:00,1.08320,1.09527,1.08038,1.09455,1.074994,1,Buy
11,2023-06-15 21:00:00+00:00,1.09444,1.09708,1.09178,1.09416,1.076795,1,Buy
12,2023-06-18 21:00:00+00:00,1.09422,1.09463,1.09072,1.09210,1.079131,1,Buy
13,2023-06-19 21:00:00+00:00,1.09226,1.09462,1.08922,1.09179,1.081196,1,Buy
...,...,...,...,...,...,...,...,...
84,2023-09-26 21:00:00+00:00,1.05698,1.05743,1.04880,1.05026,1.065352,-1,
85,2023-09-27 21:00:00+00:00,1.05074,1.05790,1.04911,1.05664,1.063134,-1,
86,2023-09-28 21:00:00+00:00,1.05616,1.06171,1.05581,1.05744,1.062362,-1,
87,2023-10-01 21:00:00+00:00,1.05720,1.05919,1.04779,1.04780,1.061455,-1,


In [23]:
# Filtering the DataFrame
buy_df = df[df['StratOrder'].isin(['Buy'])]
# Displaying the result
display(buy_df)

,time,open,high,low,close,MA10,Signal,StratOrder
9,2023-06-13 21:00:00+00:00,1.07954,1.08644,1.07740,1.08316,1.073528,1,Buy
10,2023-06-14 21:00:00+00:00,1.08320,1.09527,1.08038,1.09455,1.074994,1,Buy
11,2023-06-15 21:00:00+00:00,1.09444,1.09708,1.09178,1.09416,1.076795,1,Buy
12,2023-06-18 21:00:00+00:00,1.09422,1.09463,1.09072,1.09210,1.079131,1,Buy
13,2023-06-19 21:00:00+00:00,1.09226,1.09462,1.08922,1.09179,1.081196,1,Buy
14,2023-06-20 21:00:00+00:00,1.09164,1.09908,1.09058,1.09866,1.083424,1,Buy
15,2023-06-21 21:00:00+00:00,1.09866,1.10120,1.09485,1.09558,1.086298,1,Buy
16,2023-06-22 21:00:00+00:00,1.09584,1.09594,1.08444,1.08960,1.088044,1,Buy
17,2023-06-25 21:00:00+00:00,1.08990,1.09202,1.08874,1.09050,1.089542,1,Buy
19,2023-06-27 21:00:00+00:00,1.09604,1.09630,1.08968,1.09128,1.092694,1,Buy


In [24]:

# Find values in 'Open' and 'Close' columns in the row after 'Buy' in 'Strat Order'
buy_indices = df[df['StratOrder'] == 'Buy'].index

# Finds the next day to operate buy signal on!  Eureka! 
next_row_indices = buy_indices + 1
# Make the dataframe for next rows so we can calculate balance etc afterwards
nextday_df = df.loc[next_row_indices, ['time','open', 'close','high','low','MA10',]]
ndf = nextday_df
# Display the result DataFrame
display(ndf)

,time,open,close,high,low,MA10
10,2023-06-14 21:00:00+00:00,1.08320,1.09455,1.09527,1.08038,1.074994
11,2023-06-15 21:00:00+00:00,1.09444,1.09416,1.09708,1.09178,1.076795
12,2023-06-18 21:00:00+00:00,1.09422,1.09210,1.09463,1.09072,1.079131
13,2023-06-19 21:00:00+00:00,1.09226,1.09179,1.09462,1.08922,1.081196
14,2023-06-20 21:00:00+00:00,1.09164,1.09866,1.09908,1.09058,1.083424
15,2023-06-21 21:00:00+00:00,1.09866,1.09558,1.10120,1.09485,1.086298
16,2023-06-22 21:00:00+00:00,1.09584,1.08960,1.09594,1.08444,1.088044
17,2023-06-25 21:00:00+00:00,1.08990,1.09050,1.09202,1.08874,1.089542
18,2023-06-26 21:00:00+00:00,1.09074,1.09608,1.09766,1.09020,1.091044
20,2023-06-28 21:00:00+00:00,1.09138,1.08644,1.09414,1.08603,1.093512


Now, let's add more columns to our filtered dataframe with all subsequent rows on which the transactions occur.

In [25]:
# Create Delta column to gauge intraday change in price 
ndf['Delta']= ndf['close']-ndf['open']

# Create Range column to track distance between low and high 
ndf['Range']= ndf['high']-ndf['low']

# Create Unit Tradesize Column which is in units instead of dollars, longer explanation detailing margin, unit size, leverage etc. 
ndf['Unit Size']= 50000

# Create Tradesize 
ndf['Tradesize'] = ndf['Unit Size'] * ndf['close']

# Create Margin Used on particular trade 
ndf['Margin Used'] = .02 * ndf['Tradesize']

# Create the Strategy profit column using delta (intraday change) and unit size
ndf['StratProfit']= ndf['Delta'] * ndf['Unit Size']

# Define and Calculate Balance and Strat Initial Balance (the price at the open of first day trade)
SIB = 10000
cum_profit = ndf['StratProfit'].cumsum()
ndf['Strat_Balance'] = cum_profit  + SIB 
Strat_Balance = ndf['Strat_Balance']

# Win Loss Column added
def is_profitable(row):
    if row['StratProfit'] > 0:
        return 'Win'
    else:
        return 'Loss'

ndf['WinLoss'] = ndf.apply(is_profitable, axis=1)

# Add 'WIN' column to store positive profits
def get_win_amount(row):
    if row['StratProfit'] > 0:
        return row['StratProfit']
    else:
        return 0.00

ndf['Win'] = ndf.apply(get_win_amount, axis=1)

# Add 'Loss' column to store negative profits
def get_loss_amount(row):
    if row['StratProfit'] < 0:
        return row['StratProfit']
    else:
        return 0.00

ndf['Loss'] = ndf.apply(get_loss_amount, axis=1)

# Strategy Returns Column 
ndf['Strat_Returns_pct'] =((Strat_Balance/SIB) -1)*100
Strat_Returns = ndf['Strat_Returns_pct']


# Buy and Hold as a Benchmark 
# Create bhBalance from shares * close 

bhBalance = 50000 * ndf['close']
ndf['bhBalance']= bhBalance

# Calculate the bh drawdown
drawdown = ndf["bhBalance"].max() - ndf["bhBalance"]
ndf['bhDrawdown'] = drawdown



# Buy and Hold Strategy Pct Return and Cumulative Strategy Pct Return 

# Define initial balance and strategy balance
initial_balance = 10000  # Example initial balance
bhbalance = ndf['bhBalance']    # Example strategy balance

# Calculate strategy return
bhreturn = ((bhbalance - initial_balance) / initial_balance) 
bhreturnpct= ((bhbalance - initial_balance) / initial_balance) 


# Return is cumulative for bh strategy (buy and hold strategy)
ndf['bhreturn']=bhreturn
ndf['bhreturnpct']=bhreturnpct 

display(ndf)



,time,open,close,high,low,MA10,Delta,Range,Unit Size,Tradesize,...,StratProfit,Strat_Balance,WinLoss,Win,Loss,Strat_Returns_pct,bhBalance,bhDrawdown,bhreturn,bhreturnpct
10,2023-06-14 21:00:00+00:00,1.08320,1.09455,1.09527,1.08038,1.074994,0.01135,0.01489,50000,54727.5,...,567.5,10567.5,Win,567.5,0.0,5.675,54727.5,1471.5,4.47275,4.47275
11,2023-06-15 21:00:00+00:00,1.09444,1.09416,1.09708,1.09178,1.076795,-0.00028,0.00530,50000,54708.0,...,-14.0,10553.5,Loss,0.0,-14.0,5.535,54708.0,1491.0,4.47080,4.47080
12,2023-06-18 21:00:00+00:00,1.09422,1.09210,1.09463,1.09072,1.079131,-0.00212,0.00391,50000,54605.0,...,-106.0,10447.5,Loss,0.0,-106.0,4.475,54605.0,1594.0,4.46050,4.46050
13,2023-06-19 21:00:00+00:00,1.09226,1.09179,1.09462,1.08922,1.081196,-0.00047,0.00540,50000,54589.5,...,-23.5,10424.0,Loss,0.0,-23.5,4.240,54589.5,1609.5,4.45895,4.45895
14,2023-06-20 21:00:00+00:00,1.09164,1.09866,1.09908,1.09058,1.083424,0.00702,0.00850,50000,54933.0,...,351.0,10775.0,Win,351.0,0.0,7.750,54933.0,1266.0,4.49330,4.49330
15,2023-06-21 21:00:00+00:00,1.09866,1.09558,1.10120,1.09485,1.086298,-0.00308,0.00635,50000,54779.0,...,-154.0,10621.0,Loss,0.0,-154.0,6.210,54779.0,1420.0,4.47790,4.47790
16,2023-06-22 21:00:00+00:00,1.09584,1.08960,1.09594,1.08444,1.088044,-0.00624,0.01150,50000,54480.0,...,-312.0,10309.0,Loss,0.0,-312.0,3.090,54480.0,1719.0,4.44800,4.44800
17,2023-06-25 21:00:00+00:00,1.08990,1.09050,1.09202,1.08874,1.089542,0.00060,0.00328,50000,54525.0,...,30.0,10339.0,Win,30.0,0.0,3.390,54525.0,1674.0,4.45250,4.45250
18,2023-06-26 21:00:00+00:00,1.09074,1.09608,1.09766,1.09020,1.091044,0.00534,0.00746,50000,54804.0,...,267.0,10606.0,Win,267.0,0.0,6.060,54804.0,1395.0,4.48040,4.48040
20,2023-06-28 21:00:00+00:00,1.09138,1.08644,1.09414,1.08603,1.093512,-0.00494,0.00811,50000,54322.0,...,-247.0,10359.0,Loss,0.0,-247.0,3.590,54322.0,1877.0,4.43220,4.43220
